In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
shubhamgoel27_dermnet_path = kagglehub.dataset_download('shubhamgoel27/dermnet')

print('Data source import complete.')


Data source import complete.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import v2

import matplotlib.pyplot as plt
import numpy as np


import kagglehub
import os
from torch.utils.data import Dataset
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import DataLoader

from safetensors.torch import save_file, load_file
#from kaggle_secrets import UserSecretsClient
import wandb
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision.models import efficientnet_b3  # EfficientNet
from safetensors.torch import save_file, load_file
from collections import Counter  # Added
import json


In [3]:

import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'dl.ipynb' 
status = wandb.login(key='78fdef8c508ef3b1b92f66345a383b486d6b36f3')             #changed key, reset its
if (status):
     print('Successfully logged into W&B')
else:
     print('Unable to log into W&B')

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dl.ipynb.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kehyiqian (kehyiqian-universiti-tunku-abdul-rahman) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged into W&B


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
def save_model_and_history(model, history, save_dir="checkpoints", model_name="model", epoch=None):
    os.makedirs(save_dir, exist_ok=True)
    
    # Save model weights
    model_path = os.path.join(save_dir, f"{model_name}_weights.pth")
    torch.save(model.state_dict(), model_path)

    # Save training history
    history_path = os.path.join(save_dir, f"{model_name}_history.json")
    with open(history_path, "w") as f:
        json.dump(history, f)

    print(f"Model saved to: {model_path}")
    print(f"History saved to: {history_path}")

In [6]:
def train_one_epoch(epoch, model, train_loader, criterion, optimizer, device="cuda", log_step=20, mixup_alpha=0.1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Mixup
        if mixup_alpha > 0:
            lam = np.random.beta(mixup_alpha, mixup_alpha)
            rand_index = torch.randperm(inputs.size(0)).to(device)
            inputs = lam * inputs + (1 - lam) * inputs[rand_index]
            labels_a, labels_b = labels, labels[rand_index]
        else:
            labels_a = labels_b = labels
            lam = 1.0

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lam * criterion(outputs, labels_a) + (1 - lam) * criterion(outputs, labels_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        

        # For metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (lam * predicted.eq(labels_a).sum().item() + (1 - lam) * predicted.eq(labels_b).sum().item())
        total += labels.size(0)

        if i % log_step == 0 or i == len(train_loader) - 1:
            print(f"[Epoch {epoch+1}, Step {i+1}] train_loss: {running_loss / (i + 1):.4f}")

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    return train_loss, train_acc


In [7]:
def evaluate(model, test_loader, criterion, device="cuda"):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    with torch.inference_mode():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss += loss.item() * inputs.size(0)  # scale loss by batch size
    test_loss /= total
    test_acc = 100 * correct / total

    return test_loss, test_acc

In [8]:
#Address Class Imbalance #Focal Loss will focus on hard examples, particularly minority classes, improving overall Test Accuracy. #added label smoothing
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean', label_smoothing=0.1):   #high gamma may over-focus on hard examples, causing fluctuations.smoothen testloss and generalisation
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.reduction = reduction
        self.alpha = alpha
        self.label_smoothing = label_smoothing

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(weight=self.alpha, reduction='none', label_smoothing=self.label_smoothing)(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

In [9]:
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau

def train(model, optimizer, criterion, train_loader, test_loader, config, device="cuda", model_name="model"):

    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)
   
    best_acc = 0
    patience = 7
    patience_counter = 0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(config['num_epochs']):
        train_loss, train_acc = train_one_epoch(epoch, model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(test_loss)
        history['val_acc'].append(test_acc)

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "validation_loss": test_loss,
            "validation_acc": test_acc,
            "lr_backbone": optimizer.param_groups[0]['lr'],
            "lr_classifier": optimizer.param_groups[1]['lr']
        })


        print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.2f}%, Train Loss: {train_loss:.4f}, Validation Acc: {test_acc:.2f}%, Validation Loss: {test_loss:.4f}")

        # Save best model
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            save_file(model.state_dict(), f"best_model_epoch_{epoch+1}.safetensors")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

        scheduler.step(test_loss)

    save_model_and_history(model, history, model_name=model_name)
        

In [10]:
from torchvision.models import (
    densenet121, mobilenet_v2, efficientnet_b3, DenseNet121_Weights, MobileNet_V2_Weights, EfficientNet_B3_Weights
)
import torch.nn as nn
import torch.optim as optim

def build_model(model_name, num_classes, config, device='cuda', return_optimizer=True):
    # Select model
    if model_name == 'densenet':
        model = densenet121(weights=config['weights']).to(device)
        in_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        ).to(device)
    elif model_name == 'mobilenet':
        model = mobilenet_v2(weights=config['weights']).to(device)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features, num_classes)
        ).to(device)
    elif model_name == 'efficientnet':
        model = efficientnet_b3(weights=config['weights']).to(device)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, num_classes)
        ).to(device)
    else:
        raise ValueError("Unsupported model name")

    # Finetuning strategy
    finetuned_layers = config.get('finetuned_layers', 'all')

    if finetuned_layers == 'classifier':
        for param in model.parameters():
            param.requires_grad = False
        for param in model.classifier.parameters():
            param.requires_grad = True
    elif finetuned_layers == 'features_last' and model_name == 'efficientnet':
        for param in model.parameters():
            param.requires_grad = False
        for param in model.features[-1].parameters():
            param.requires_grad = True
        for param in model.classifier.parameters():
            param.requires_grad = True
    elif finetuned_layers == 'all':
        for param in model.parameters():
            param.requires_grad = True
    else:
        for name, param in model.named_parameters():
            if not any(name.startswith(layer) for layer in config['finetuned_layers']):
                param.requires_grad = False

    # Print model info
    print(f"Model           : {model_name}")
    print(f"Weights         : {config['weights']}")
    print(f"Finetuned layers: {finetuned_layers}\n")

    # Optimizer
    if not return_optimizer:
        return model

    if hasattr(model, 'features') and hasattr(model, 'classifier'):
        optimizer = torch.optim.AdamW([
            {"params": model.features.parameters(), "lr": config.get("backbone_lr",config['lr_features'])},
            {"params": model.classifier.parameters(), "lr": config.get("classifier_lr",config['lr_classifier'])}
        ], weight_decay=config.get("weight_decay", 1e-3))
    else:
        raise ValueError("Model must have 'features' and 'classifier' attributes.")

  

    return model, optimizer


In [11]:
path = kagglehub.dataset_download("shubhamgoel27/dermnet") # directly download the dataset from kaggle
dataset_path = os.listdir(path)

train_path = os.path.join(path, 'train')   #train folder
test_path = os.path.join(path, 'test')     #test folder
# List class names from the 'train' folder
class_names = [folder for folder in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, folder))]
num_classes = len(class_names)
print(f'Class names found in the training set: \n{class_names} \n')
print(f'{num_classes = } \n')

class_names = [folder for folder in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, folder))]
num_classes = len(class_names)
print(f'Class names found in the testing set: \n{class_names} \n')
print(f'{num_classes = } \n')

Class names found in the training set: 
['Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Acne and Rosacea Photos', 'Systemic Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Vascular Tumors', 'Urticaria Hives', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Psoriasis pictures Lichen Planus and related diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Scabies Lyme Disease and other Infestations and Bites', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Herpes HPV and other STDs Photos', 'Seborrheic Keratoses and other Benign Tumors', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Vasculitis Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Warts Molluscum and other Viral Infections'] 

num_classes = 23 

Class names found in the test

In [12]:
class SkinDiseaseDataset(Dataset):
    def __init__(self, path, transform=None, selected_classes=None):
        self.path = path
        self.transform = transform
        self.classes = []            # filtered class folder names
        self.class_to_idx = {}       # mapping: class name -> label index
        self.images_path = []
        self.targets = []

        for i, cls in enumerate(sorted(os.listdir(path))):
            if selected_classes is not None and cls not in selected_classes:
                continue
            self.classes.append(cls)
            self.class_to_idx[cls] = len(self.class_to_idx)  # re-index
            cls_path = os.path.join(path, cls)
            for filename in os.listdir(cls_path):
                file_path = os.path.join(cls_path, filename)
                self.images_path.append(file_path)
                self.targets.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image_path = self.images_path[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, label


In [13]:

transform_train = v2.Compose([
    v2.ToImage(), # Convert input to PIL Image if needed
    v2.Resize((224,224)), # Resize to 512x512, 224x224
    v2.RandomHorizontalFlip(), # Randomly flip horizontally
    v2.RandomVerticalFlip(p=0.5),  # Added vertical flip
    v2.RandomRotation(degrees=10),                                                                # Reduced to 5 degrees from 10

    
    # v2.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),  # Soften translation and scaling
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Soften color jitter
    # v2.RandomErasing(p=0.3, scale=(0.02, 0.1), ratio=(0.3, 3.3)),  # Soften random erasing
    v2.ToDtype(torch.float32, scale=True), # Scale to [0,1] and make float32
    v2.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225]),# Normalization
])

transform_test = v2.Compose([
    v2.ToImage(),                         
    v2.Resize((224,224)),                 
    v2.ToDtype(torch.float32, scale=True), 
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [14]:
train_dataset = SkinDiseaseDataset(train_path, transform_train)
test_dataset = SkinDiseaseDataset(test_path, transform_test)


In [15]:
#15
# use class weights to distru
# Compute class weights
def check_class_distribution(dataset):
    labels = [label for _, label in dataset]
    class_counts = Counter(labels)
    print("Class distribution:")
    for class_idx, count in class_counts.items():
        class_name = class_names[class_idx]
        print(f"{class_name}: {count} samples")
    return class_counts

class_counts = check_class_distribution(train_dataset)
total_samples = sum(class_counts.values())
class_weights = torch.tensor([total_samples / (num_classes * class_counts[i]) for i in range(num_classes)], dtype=torch.float32)
smoothing_factor = 0.5
uniform_weights = torch.ones(num_classes, dtype=torch.float32)
class_weights = smoothing_factor * class_weights + (1 - smoothing_factor) * uniform_weights         #Label smoothing can prevent the model from becoming overconfident, improving generalization.
class_weights = class_weights.to(device)


# Update loss function with class weights
loss_function = FocalLoss(alpha=class_weights, gamma=3.0, reduction='mean', label_smoothing=0.1)   #label smoothing
# loss_function = nn.CrossEntropyLoss()

Class distribution:
Light Diseases and Disorders of Pigmentation: 840 samples
Lupus and other Connective Tissue diseases: 1149 samples
Acne and Rosacea Photos: 489 samples
Systemic Disease: 448 samples
Poison Ivy Photos and other Contact Dermatitis: 288 samples
Vascular Tumors: 1235 samples
Urticaria Hives: 404 samples
Atopic Dermatitis Photos: 239 samples
Bullous Disease Photos: 405 samples
Hair Loss Photos Alopecia and other Hair Diseases: 568 samples
Tinea Ringworm Candidiasis and other Fungal Infections: 420 samples
Psoriasis pictures Lichen Planus and related diseases: 463 samples
Melanoma Skin Cancer Nevi and Moles: 1040 samples
Nail Fungus and other Nail Disease: 260 samples
Scabies Lyme Disease and other Infestations and Bites: 1405 samples
Eczema Photos: 431 samples
Exanthems and Drug Eruptions: 1371 samples
Herpes HPV and other STDs Photos: 606 samples
Seborrheic Keratoses and other Benign Tumors: 1300 samples
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions

In [16]:
#16
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(splitter.split(train_dataset.images_path, train_dataset.targets))

print('Number of training samples:', len(train_idx), "| indices:", train_idx)
print('Number of val samples     :', len(val_idx), "| indices:", val_idx)

Number of training samples: 12445 | indices: [ 8880 14613  9269 ... 14918  1808  2715]
Number of val samples     : 3112 | indices: [  474 14781   392 ...  8378 10909  8603]


In [17]:
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [19]:
#19
x_batch, y_batch = next(iter(train_loader))
print(f'{x_batch.shape = }')
print(f'{y_batch.shape = }')


x_batch.shape = torch.Size([32, 3, 224, 224])
y_batch.shape = torch.Size([32])


# densenet

In [20]:
'''
model_type = 'densenet'
config =  {
    'weights'         : 'IMAGENET1K_V1',
    'finetuned_layers': 'all',
    'lr_features'     : 1e-4,
    'lr_classifier'   : 5e-4,
    'num_epochs'      : 30
}
'''

"\nmodel_type = 'densenet'\nconfig =  {\n    'weights'         : 'IMAGENET1K_V1',\n    'finetuned_layers': 'all',\n    'lr_features'     : 1e-4,\n    'lr_classifier'   : 5e-4,\n    'num_epochs'      : 30\n}\n"

In [21]:
#wandb.finish()

In [22]:
'''
wandb.init(
    project='DL_groupAssigment',
    name='All - output-test1',
    config =  {
    'weights'         : 'IMAGENET1K_V1',
    'finetuned_layers': 'all',
    'lr_features'     : 1e-4,
    'lr_classifier'   : 5e-4,
    'num_epochs'      : 30
    },
)

'''

"\nwandb.init(\n    project='DL_groupAssigment',\n    name='All - output-test1',\n    config =  {\n    'weights'         : 'IMAGENET1K_V1',\n    'finetuned_layers': 'all',\n    'lr_features'     : 1e-4,\n    'lr_classifier'   : 5e-4,\n    'num_epochs'      : 30\n    },\n)\n\n"

# **DENSENET **

In [23]:
#model_densenet, optimizer = build_model(model_type, num_classes, config, device)
#train(model_densenet, optimizer, loss_function, train_loader, val_loader, config, device, model_name = "model_densenet")

In [24]:
#cell 23

# **MOBILENET **

In [25]:
# model_type = 'mobilenet'
# config =  {
#     'weights'         : 'IMAGENET1K_V1',
#     'finetuned_layers': 'all',
#     'lr'              : 3e-4,
#     'num_epochs'      : 30
# }

In [26]:
# model_mobilenet, optimizer = build_model(model_type, num_classes, config, device)
# train(model_mobilenet, optimizer, loss_function, train_loader, val_loader, config, device)

# **EfficientNet **

In [27]:
val_dataset = SkinDiseaseDataset(train_path, transform_test)
val_loader = DataLoader(val_dataset, batch_size=32, sampler=val_sampler)

In [28]:
# EfficientNet Configuration and Training
config_efficientnet = {
    'num_epochs': 6,
    'weights': 'IMAGENET1K_V1',
    'finetuned_layers': 'classifier',
    'batch_size': 32,
    'lr_features': 0.0001,  
    'lr_classifier': 0.0005,  
}

# Training function for EfficientNet
def run_efficientnet(config, train_loader, val_loader, num_classes, device):
    print("\nTraining EfficientNet")
    model_type = 'efficientnet'

    # Initialize WandB
    wandb.init(
        project='DL_groupAssigment',
        name='EfficientNet_Training',
        config=config
    )

    
    # Stage 1: Fine-tune classifier
    config['finetuned_layers'] = 'classifier'
    config['num_epochs'] = 6
    print("Stage 1: Fine-tuning classifier")
    model, optimizer = build_model(model_type, num_classes, config, device)
    train(model, optimizer, loss_function, train_loader, val_loader, config, device, model_name="efficientnet")

    # Log Stage 1 results
    val_loss, val_acc = evaluate(model, val_loader, loss_function, device)
    wandb.log({"stage": "Stage 1", "val_loss": val_loss, "val_acc": val_acc})

    # Clear GPU memory
    torch.cuda.empty_cache()

    # Stage 2: Fine-tune all layers
    print("Stage 2: Fine-tuning all layers")
    config['finetuned_layers'] = 'all'
    config['num_epochs'] = 24  # Total 30 epochs
    config['lr_features'] = 1e-4
    config['lr_classifier'] = 0.0001

    for param in model.parameters():
        param.requires_grad = True
    
    # Reinitialize optimizer with new learning rates
    optimizer = torch.optim.AdamW([
        {"params": model.features.parameters(), "lr": config['lr_features']},
        {"params": model.classifier.parameters(), "lr": config['lr_classifier']}
    ], weight_decay=config.get("weight_decay", 1e-3))

    train(model, optimizer, loss_function, train_loader, val_loader, config, device, model_name="efficientnet")

    # Log Stage 2 results
    test_loss, test_acc = evaluate(model, test_loader, loss_function, device)
    wandb.log({"stage": "Stage 2", "test_loss": test_loss, "test_acc": test_acc})
    print(f"EfficientNet Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.2f}%")
    save_file(model.state_dict(), "efficientnet_b3_all_lr0.0001.safetensors")
    print("Model saved.")

    return model



# Run EfficientNet
model_efficientnet = run_efficientnet(config_efficientnet, train_loader, val_loader, num_classes, device)
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.



Training EfficientNet


Stage 1: Fine-tuning classifier
Model           : efficientnet
Weights         : IMAGENET1K_V1
Finetuned layers: classifier

[Epoch 1, Step 1] train_loss: 2.4479
[Epoch 1, Step 21] train_loss: 2.7361
[Epoch 1, Step 41] train_loss: 2.7625
[Epoch 1, Step 61] train_loss: 2.7209
[Epoch 1, Step 81] train_loss: 2.7240
[Epoch 1, Step 101] train_loss: 2.6927
[Epoch 1, Step 121] train_loss: 2.6733
[Epoch 1, Step 141] train_loss: 2.6529
[Epoch 1, Step 161] train_loss: 2.6377
[Epoch 1, Step 181] train_loss: 2.6286
[Epoch 1, Step 201] train_loss: 2.6217
[Epoch 1, Step 221] train_loss: 2.6174
[Epoch 1, Step 241] train_loss: 2.6084
[Epoch 1, Step 261] train_loss: 2.5951
[Epoch 1, Step 281] train_loss: 2.5898
[Epoch 1, Step 301] train_loss: 2.5709
[Epoch 1, Step 321] train_loss: 2.5628
[Epoch 1, Step 341] train_loss: 2.5622
[Epoch 1, Step 361] train_loss: 2.5554
[Epoch 1, Step 381] train_loss: 2.5416
[Epoch 1, Step 389] train_loss: 2.5361
[Epoch 1] Train Acc: 19.32%, Train Loss: 2.5361, Validation Ac

# Evaluation

In [29]:
#### report
from sklearn.metrics import classification_report, precision_recall_curve, average_precision_score, confusion_matrix
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from typing import Tuple, Dict
import torchvision.models as models


def load_model(model_name: str, num_classes: int, weights_path: str, device: str) -> nn.Module:
    if model_name == 'densenet':
        model = models.densenet121(weights=None)
        in_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    elif model_name == 'mobilenet':
        model = models.mobilenet_v2(weights=None)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features, num_classes)
        )
    elif model_name == 'efficientnet':
        model = models.efficientnet_b3(weights=None)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features, num_classes)
        )
    else:
        raise ValueError("Unsupported model type")

    model.load_state_dict(torch.load(weights_path, map_location=device, weights_only=True))

    return model.to(device)

In [30]:
#model_densenet = load_model(model_name='densenet', num_classes= num_classes, weights_path='checkpoints/model_densenet_weights.pth', device=device)
model_efficientnet = load_model(model_name='efficientnet', num_classes=num_classes, weights_path='efficientnet_b3_all_lr0.0001.safetensors', device=device)

UnpicklingError: Weights only load failed. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 112

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.

In [ ]:
def load_history_json(history_path: str) -> Dict:
    with open(history_path, 'r') as f:
        history = json.load(f)
    return history

In [ ]:
history = load_history_json('checkpoints/model_densenet_history.json')

In [ ]:
test_loss, test_acc = evaluate(model_densenet, test_loader, loss_function, device)
print(f"Loaded Model Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.2f}%")

In [ ]:

def get_classification_report(model, test_loader, class_names, device):
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    report = classification_report(y_true, y_pred, target_names=class_names, digits=3)
    return report


In [ ]:
report = get_classification_report(model_efficientnet, test_loader, class_names, device)
print(report)

In [ ]:

def plot_weighted_avg_precision_recall_curve(model, test_loader, class_names, device):
    model.eval()
    y_true = []
    y_scores = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            y_scores.extend(probs)
            y_true.extend(labels.cpu().numpy())

    y_scores = np.array(y_scores)
    y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))

    precision, recall, _ = precision_recall_curve(y_true_bin.ravel(), y_scores.ravel())
    ap = average_precision_score(y_true_bin, y_scores, average='weighted')

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label=f'Micro-Averaged AP = {ap:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Weighted-Averaged Precision-Recall Curve')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()


In [ ]:
plot_weighted_avg_precision_recall_curve(model_densenet, test_loader, class_names, device)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, test_loader, class_names, device):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
plot_confusion_matrix(model_densenet, test_loader, class_names, device)

In [ ]:
#!/usr/bin/env python3
# inference_mobilenetv2.py - Test script for MobileNetV2 model

import sys
import torch
import argparse
from torchvision.models import MobileNet_V2_Weights
from PIL import Image
from torchvision import transforms

# Import the build_model function from your module
# Assuming the build_model function is in a file called models.py
# If it's in a different file, change the import accordingly
from models import build_model

# ——— CONFIGURATION ———
MODEL_PATH = "mbnet_dermnet_v2.pt"
NUM_CLASSES = 23
CLASS_NAMES = [
    "Light Diseases and Disorders of Pigmentation",
    "Lupus and other Connective Tissue diseases",
    "Acne and Rosacea Photos",
    "Systemic Disease",
    "Poison Ivy Photos and other Contact Dermatitis",
    "Vascular Tumors",
    "Urticaria Hives",
    "Atopic Dermatitis Photos",
    "Bullous Disease Photos",
    "Hair Loss Photos Alopecia and other Hair Diseases",
    "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Psoriasis pictures Lichen Planus and related diseases",
    "Melanoma Skin Cancer Nevi and Moles",
    "Nail Fungus and other Nail Disease",
    "Scabies Lyme Disease and other Infestations and Bites",
    "Eczema Photos",
    "Exanthems and Drug Eruptions",
    "Herpes HPV and other STDs Photos",
    "Seborrheic Keratoses and other Benign Tumors",
    "Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions",
    "Vasculitis Photos",
    "Cellulitis Impetigo and other Bacterial Infections",
    "Warts Molluscum and other Viral Infections"
]

IMAGE_SIZE = 224  # match your training size

def load_model(model_path, device='cpu'):
    """
    Load a saved MobileNetV2 model
    """
    # Define model configuration
    config = {
        'weights': None,  # We'll load from saved weights, not pretrained
        'lr': 1e-4,
        'weight_decay': 1e-4,
        'finetuned_layers': 'all'
    }
    
    # Build the model architecture using your function
    model = build_model('mobilenet', NUM_CLASSES, config, device=device, return_optimizer=False)
    
    # Load the saved weights
    checkpoint = torch.load(model_path, map_location=device)
    
    # Handle different saving formats
    if isinstance(checkpoint, dict) and "state_dict" in checkpoint:
        model.load_state_dict(checkpoint["state_dict"])
    elif isinstance(checkpoint, dict) and "model" in checkpoint:
        model.load_state_dict(checkpoint["model"])
    else:
        try:
            model.load_state_dict(checkpoint)
        except RuntimeError as e:
            print(f"Error loading model: {e}")
            # If it's the entire model object
            if isinstance(checkpoint, torch.nn.Module):
                model = checkpoint
    
    model.eval()
    return model

def get_transforms():
    """
    Create image preprocessing transformations
    """
    return transforms.Compose([
        transforms.Resize(int(IMAGE_SIZE * 1.14)),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ])

def predict(model, image_path, transform, topk=5, device='cpu'):
    """
    Make predictions on an image
    """
    # Open and preprocess the image
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    
    # Make prediction
    with torch.no_grad():
        logits = model(x)
        probs = torch.nn.functional.softmax(logits[0], dim=0)
    
    # Get top k predictions
    top_probs, top_idxs = probs.topk(topk)
    
    print(f"\nTop {topk} predictions for {image_path}:")
    for p, idx in zip(top_probs, top_idxs):
        print(f"  {CLASS_NAMES[idx]} — {p.item():.4f}")
    
    return top_probs, top_idxs

def main():
    parser = argparse.ArgumentParser(description='Inference with MobileNetV2 model')
    parser.add_argument('image_path', type=str, help='Path to input image')
    parser.add_argument('--model', type=str, default=MODEL_PATH, help='Path to model weights')
    parser.add_argument('--topk', type=int, default=5, help='Show top k predictions')
    parser.add_argument('--device', type=str, default='cpu', help='Device to run inference on (cpu/cuda)')
    args = parser.parse_args()
    
    # Check if CUDA is available
    if args.device == 'cuda' and not torch.cuda.is_available():
        print("CUDA is not available. Using CPU instead.")
        args.device = 'cpu'
    
    print(f"Loading model from {args.model}...")
    model = load_model(args.model, device=args.device)
    
    transform = get_transforms()
    
    print(f"Running inference on {args.image_path}...")
    predict(model, args.image_path, transform, args.topk, device=args.device)

if _name_ == "_main_":
    main()


In [ ]:
# models.py - Contains the model building functions

from torchvision.models import (
    densenet121, mobilenet_v2, efficientnet_b3, DenseNet121_Weights, MobileNet_V2_Weights, EfficientNet_B3_Weights
)
import torch.nn as nn
import torch.optim as optim

def build_model(model_name, num_classes, config, device='cuda', return_optimizer=True):
    """
    Build and configure a neural network model.
    
    Args:
        model_name (str): Name of the model ('densenet', 'mobilenet', or 'efficientnet')
        num_classes (int): Number of output classes
        config (dict): Configuration with weights, learning rate, etc.
        device (str): Device to put the model on ('cuda' or 'cpu')
        return_optimizer (bool): Whether to return optimizer along with the model
        
    Returns:
        model or (model, optimizer) depending on return_optimizer
    """
    # Select model
    if model_name == 'densenet':
        model = densenet121(weights=config['weights']).to(device)
        in_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        ).to(device)
    elif model_name == 'mobilenet':
        model = mobilenet_v2(weights=config['weights']).to(device)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features, num_classes)
        ).to(device)
    elif model_name == 'efficientnet':
        model = efficientnet_b3(weights=config['weights']).to(device)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.65),
            nn.Linear(in_features, num_classes)
        ).to(device)
    else:
        raise ValueError(f"Unsupported model name: {model_name}")
        
    # Finetuning strategy
    finetuned_layers = config.get('finetuned_layers', 'all')
    if finetuned_layers == 'classifier':
        for param in model.parameters():
            param.requires_grad = False
        for param in model.classifier.parameters():
            param.requires_grad = True
    elif finetuned_layers == 'features_last' and model_name == 'efficientnet':
        for param in model.parameters():
            param.requires_grad = False
        for param in model.features[-1].parameters():
            param.requires_grad = True
        for param in model.classifier.parameters():
            param.requires_grad = True
    elif finetuned_layers == 'all':
        for param in model.parameters():
            param.requires_grad = True
    else:
        for name, param in model.named_parameters():
            if not any(name.startswith(layer) for layer in config['finetuned_layers']):
                param.requires_grad = False
                
    # Print model info
    print(f"Model           : {model_name}")
    print(f"Weights         : {config['weights']}")
    print(f"Finetuned layers: {finetuned_layers}\n")
    
    # Optimizer
    if not return_optimizer:
        return model
        
    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=config['lr'],
        weight_decay=config.get('weight_decay', 1e-4)
    )
    
    return model, optimizer
